# Cvičenie 6: Neurónové siete v PyTorch

Minulý týždeň ste už videli príklad implementácie neurónovej siete pomocou knižnice PyTorch. Na dnešnom cvičení sa pozrieme na to, ako presne funguje Autograd v PyTorch. Aby ste úspešne zvládli cvičenie, potrebujete mať nainštalovanú knižnicu PyTorch.

## 1. Výpočty v PyTorch

Na začiatok zadefinujeme jednoduchú operáciu sčítania pomocou PyTorch:

![](https://github.com/DominikVranay/neural-networks-course/blob/master/labs/sources/lab06/6.1-addition.png?raw=1)

In [1]:
import torch

# create the nodes in the graph, and initialize values
a = torch.tensor(13)
b = torch.tensor(37)

# add together the two values
c = torch.add(a, b)
print(c)

tensor(50)


Výstup výpočtu je tensor s hodnotou 50: `tensor(50)`. PyTorch podporuje tensory s hodnotami (môžu byť konštanty alebo premenné) a s výpočtami.

Na základe predošlého príkladu vytvorte viac zložitý graf:

![](https://github.com/DominikVranay/neural-networks-course/blob/master/labs/sources/lab06/6.2-complicated-graph.png?raw=1)

In [2]:
# create the nodes in the graph, and initialize values
a = torch.tensor(2.5)
b = torch.tensor(6.5)

c = a + b
d = torch.sub(b, 1) # torch.add(b, -1)
e = torch.mul(c, d)

print(e)

tensor(49.5000)


## 2. Neurón v PyTorch

Neurón predstavuje základný výpočtový prvok neurónových sietí, ktorý pozostáva zo vstupov, synaptických váh, váženej sumy, aktivačnej funkcie a výstupnej funckie. V tomto kroku implementujeme jednoduchý neurón pomocou základných metód PyTorch, aby ste videli, ako tieto výpočty fungujú na úrovni s nižšou abstrakciou.

![](https://github.com/DominikVranay/neural-networks-course/blob/master/labs/sources/lab06/6.3-perceptron.png?raw=1)

In [3]:
# simple perceptron with two input nodes
def my_neuron(x):
    # define some arbitrary weights for the two input values
    W = torch.tensor([[3, -2]], dtype=torch.float32)

    # define the bias of the neuron
    b = 1
    
    # compute weighted sum (hint: check out torch.matmul)
    z = torch.matmul(x, W.T) + b

    # apply the sigmoid activation function (hint: use torch.sigmoid)
    output = torch.sigmoid(z)

    return output

sample_input = torch.tensor([[-1, 2]], dtype=torch.float32)

result = my_neuron(sample_input)
print(result)

tensor([[0.0025]])


## 3. Plne prepojené vrstvy v PyTorch

Ak chceme vytvoriť neurónovú sieť, jednoduché neuróny musíme usporiadať do tzv. vrstiev. Oproti predošlému príkladu musíme tiež upraviť váhy tak, aby boli aktualizovateľné. V neurónových sieťach sa najčastejšie používajú plne prepojené vrstvy (*fully connected* alebo *linear*).

In [4]:
# x: input values
# n_in: number of input nodes
# n_out: number of output nodes
def my_dense_layer(x, n_in, n_out):
    # define variable weights as a matrix and biases
    # initialize weights for one
    # initialize biases for zero
    W = torch.rand((n_in, n_out), requires_grad=True)
    b = torch.zeros((1, n_out), requires_grad=True)
    
    # compute weighted sum (hint: check out torch.matmul)
    z = torch.matmul(x, W) + b

    # apply the sigmoid activation function (hint: use torch.sigmoid)
    output = torch.sigmoid(z)

    return output, W

Ako aj pred tým, naše riešenie vieme otestovať zadaním ľubovoľných hodnôt (s dodržaním počtu vstupných a výstupných neurónov).

In [5]:
sample_input = torch.tensor([[1, 2.]])
out, W = my_dense_layer(sample_input, n_in=2, n_out=3)
out.sum().backward()
print(out)
print(W.grad)

tensor([[0.8900, 0.8228, 0.7687]], grad_fn=<SigmoidBackward0>)
tensor([[0.0979, 0.1458, 0.1778],
        [0.1957, 0.2916, 0.3556]])


Podobne by sme postupovali pri definícii neurónovej siete s viacerými vrstvami, vidíme ale, že takýto zápis je zdĺhavý a neprehľadný. Práve preto bola vyvinutá knižnica `torch.nn`, ktorá obsahuje už naprogramované základné vrstvy pomocou jazyka C++.

## 4. Definícia a trénovanie neurónovej siete pomocou torch.nn

Pre dnešnú ukážku môžete používať ľubuvoľný dataset, odporúčame ale použiť niektorý dataset pre klasifikáciu z [tohto repozitára](https://archive.ics.uci.edu/ml/datasets.php).

Ako sme už videli minulý týždeň, v PyTorch potrebujeme vytvoriť sekvenčný model priamo s vrstvami, následne vytvoriť optimizátor a nakoniec natrénovať. Dnes ukážeme trocha iný prístup, kde najprv zadefinujete triedu pre model s vrstvami a prepojíte ich vo forward funkcii. Oba prístupy sú rovnocenné a môžete ich používať ľubovoľne. Najčastejšie sa používa vlastná trieda modelu, keďže je oveľa jednoduchšie manipulovať výstup vrstiev a vytvoriť viac vetiev.

In [6]:
from torch import nn

class Model(nn.Module):
    def __init__(self, dims=[4, 12, 6, 3]):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(dims[0], dims[1])
        self.linear2 = nn.Linear(dims[1], dims[2])
        self.linear3 = nn.Linear(dims[2], dims[3])
        self.relu = nn.ReLU(inplace=True)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.tanh(x)
        x = self.linear2(x)
        x = self.tanh(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x

model = Model()
print(model)

Model(
  (linear1): Linear(in_features=4, out_features=12, bias=True)
  (linear2): Linear(in_features=12, out_features=6, bias=True)
  (linear3): Linear(in_features=6, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
  (tanh): Tanh()
  (sigmoid): Sigmoid()
)


Užitočná metóda je metóda `print`, ktorá vám vypíše informácie o definovanej sieti.

Ak ste spokojní s topológiou siete, môžete vytvorit optimalizator a loss funkciu.

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
for params in model.parameters():
    print(params)

Parameter containing:
tensor([[-0.4668,  0.0452, -0.4918, -0.3462],
        [-0.3381, -0.1159,  0.1185,  0.4544],
        [-0.4656,  0.0897,  0.1166,  0.3414],
        [ 0.2769, -0.2068,  0.4797, -0.2623],
        [ 0.3770,  0.3966,  0.1118, -0.1879],
        [ 0.2787, -0.2082, -0.0675,  0.0884],
        [-0.0900, -0.2611, -0.3587,  0.2395],
        [-0.2356,  0.1900, -0.1151, -0.2405],
        [ 0.4786,  0.0169,  0.2295, -0.0773],
        [-0.1958, -0.3778,  0.4924,  0.1690],
        [-0.0637,  0.2946, -0.1608, -0.4321],
        [ 0.2398, -0.4097, -0.0168, -0.0667]], requires_grad=True)
Parameter containing:
tensor([ 0.0250, -0.0612, -0.0306, -0.3166,  0.1789,  0.1057, -0.4297,  0.2030,
         0.3223,  0.0424,  0.2716,  0.4536], requires_grad=True)
Parameter containing:
tensor([[-0.1627,  0.2177, -0.2726,  0.1958, -0.1794,  0.2339, -0.1963, -0.2041,
          0.1606, -0.2148,  0.0428,  0.2131],
        [-0.2479, -0.0672,  0.1233,  0.1330,  0.1589,  0.2081, -0.1942, -0.1495,
        

Ako Dataset si zvolíme Iris z minulých cvičené a použijeme už známy training loop

In [8]:
import pandas as pd
dataset = pd.read_csv('iris.csv')
X = dataset.iloc[:, :4].values
y = dataset.iloc[:, -1].values

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# transform string labels into number values 0, 1, 2
Y = encoder.fit_transform(y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)

In [9]:
for epoch in range(1000):
    optimizer.zero_grad()
    preds = model(X_train)
    loss = criterion(preds, y_train)
    loss.backward()
    optimizer.step()
    if epoch%100 == 0:
        print("Loss:", loss.detach().item(), "accuracy:", (y_train== preds.argmax(-1)).sum().item()/len(y_train))

Loss: 1.098787546157837 accuracy: 0.325
Loss: 0.702217698097229 accuracy: 0.9416666666666667
Loss: 0.5998117923736572 accuracy: 0.9833333333333333
Loss: 0.5823734998703003 accuracy: 0.9916666666666667
Loss: 0.5743486881256104 accuracy: 0.9916666666666667
Loss: 0.5694678425788879 accuracy: 0.9916666666666667
Loss: 0.5663651823997498 accuracy: 0.9916666666666667
Loss: 0.5644932985305786 accuracy: 0.9916666666666667
Loss: 0.5633373260498047 accuracy: 0.9916666666666667
Loss: 0.5625829696655273 accuracy: 0.9916666666666667


Ak chcete natrénovanú sieť používať, potrebujete zavolať model:

In [10]:
y_pred = model(X_test)

**Poznámka**: ukážkové riešenie úloh s PyTorch nájdete [tu](https://github.com/DominikVranay/neural-networks-course/blob/1bc027f84dbcfb48e49d05122c68512feacd3a1a/labs/sources/lab06/lab6-tensorflow-solution.py)).